In [1]:
# import pdb; pdb.set_trace()
# import pudb; pudb.set_trace()

# df.style.background_gradient(
#     cmap='coolwarm', 
#     axis=None, 
#     vmin=-1, 
#     vmax=1
# )  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG'

# pd.set_option('precision', 2)

# .plot_bokeh(sizing_mode="scale_height")
# .plot_bokeh(kind='hist', bins=1000, sizing_mode="scale_height", xlim=[0, .1])

# Imports

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.set_option('float_format', '{:f}'.format)
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', 99)

import matplotlib as mpl
from matplotlib import style
style.use('fivethirtyeight')
mpl.rcParams['figure.figsize'] = [20, 7]

In [3]:
import datetime as dt
import pickle
import numpy as np
import pandas as pd
import pandas_bokeh
pandas_bokeh.output_file("tmp/bokeh_output.html")
import ray

from data_layer import arrow_dataset, storage_adaptor
from tick_filter import streaming_tick_filter
from tick_sampler import streaming_tick_sampler, daily_stats
from workflows import sampler_task, sampler_flow
from utilities import date_fu, project_globals as g
from data_layer import storage_adaptor, fsspec_factory, data_access, arrow_dataset


config = {
    'meta': {
        'symbol': 'EGO',
        'symbol_list': ['GFI','GLD','GORO','GSS'],
        'start_date': '2019-01-01',
        'end_date': '2019-04-08',
        'config_id': 'renko_v1',
        'presist_destination': 'remote',
        'ray_on': True,
    },
    'filter': {
        'mad_value_winlen': 22,
        'mad_deviation_winlen': 1111,
        'mad_k': 17,
        'jma_winlen': 7,
        'jma_power': 2,
    },
    'sampler': {
        'renko_return': 'price_jma_return',
        'renko_size': 0.1,  # for simple runs
        'renko_reveral_multiple': 2,
        'renko_range_frac': 22,
        'renko_range_min_pct_value': 0.03,  # % of symbol value enforced as min renko size
        'max_duration_td': dt.timedelta(minutes=33),
        'min_duration_td': dt.timedelta(seconds=33),
        'min_tick_count': 33,
        'add_label': True,
        'reward_ratios': list(np.arange(2, 11, 0.5)),
    }
}

prefix_1 = f"/tick_samples/{config['meta']['config_id']}/bar_date"

prefix_2 = f"/tick_samples/{config['meta']['config_id']}/bars_df"

prefix_3 = '/data/trades'

In [5]:
data_access.list(prefix_1, 'GLD')

['2019-01-02',
 '2019-01-03',
 '2019-01-04',
 '2019-01-07',
 '2019-01-08',
 '2019-01-09',
 '2019-01-10',
 '2019-01-11',
 '2019-01-14',
 '2019-01-15',
 '2019-01-16',
 '2019-01-17',
 '2019-01-18',
 '2019-01-22',
 '2019-01-23',
 '2019-01-24',
 '2019-01-25',
 '2019-01-28',
 '2019-01-29',
 '2019-01-30',
 '2019-01-31',
 '2019-02-01',
 '2019-02-04',
 '2019-02-05',
 '2019-02-06',
 '2019-02-07',
 '2019-02-08',
 '2019-02-11',
 '2019-02-12',
 '2019-02-13',
 '2019-02-14',
 '2019-02-15',
 '2019-02-19',
 '2019-02-20',
 '2019-02-21',
 '2019-02-22',
 '2019-02-25',
 '2019-02-26',
 '2019-02-27',
 '2019-02-28',
 '2019-03-01',
 '2019-03-04',
 '2019-03-05',
 '2019-03-06',
 '2019-03-07',
 '2019-03-08',
 '2019-03-11',
 '2019-03-12',
 '2019-03-13',
 '2019-03-14',
 '2019-03-15',
 '2019-03-18',
 '2019-03-19',
 '2019-03-20',
 '2019-03-21',
 '2019-03-22',
 '2019-03-25',
 '2019-03-26',
 '2019-03-27',
 '2019-03-28',
 '2019-03-29',
 '2019-04-01']

# Sample tick dates

In [4]:
ray.init(dashboard_host='0.0.0.0', dashboard_port=1111, ignore_reinit_error=True)

2021-06-03 13:36:59,143	INFO services.py:1267 -- View the Ray dashboard at http://0.0.0.0:1111


{'node_ip_address': '192.168.1.66',
 'raylet_ip_address': '192.168.1.66',
 'redis_address': '192.168.1.66:6379',
 'object_store_address': '/tmp/ray/session_2021-06-03_13-36-56_820659_14676/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-06-03_13-36-56_820659_14676/sockets/raylet',
 'webui_url': '0.0.0.0:1111',
 'session_dir': '/tmp/ray/session_2021-06-03_13-36-56_820659_14676',
 'metrics_export_port': 52270,
 'node_id': '597860dcd086d70ad3bfa8c1ece955fec72930afacaa8867cceeb090'}

In [15]:
ray.shutdown()

In [5]:
out = sampler_flow.run(config)

(pid=14695) running GFI 2019-04-08
(pid=14695) running GLD 2019-04-08
(pid=14695) running GORO 2019-04-08
(pid=14695) running GSS 2019-04-08


In [ ]:
n = 1
print(bar_dates[n]['date'])
# bar_dates[n]['ticks_df'][['price', 'price_jma']].plot_bokeh(sizing_mode="scale_height")
bar_dates[n]['ticks_df'][['price', 'price_jma']].plot()

In [ ]:
bar_dates[n]['ticks_df'][['price_jma']].plot()

In [ ]:
bar_dates[n]['ticks_df'].status.value_counts() / len(bar_dates[n]['ticks_df'])

In [ ]:
bar_dates[n]['bars_df'][['price_low', 'price_high', 'price_close']].plot()

# Arrow dataset

In [ ]:
ds = arrow_dataset.get_dataset('GLD', prefix_2, fs_type='remote', schema=bar_date_schema)

In [ ]:
ds = arrow_dataset.get_dataset('GLD', prefix_2, fs_type='remote')

In [ ]:
bar_date_schema = ds.schema

In [ ]:
df = ds.to_table().to_pandas()

In [ ]:
df[['price_close','price_vwap']].plot_bokeh(sizing_mode="scale_height")

In [ ]:
abs(df.label_rrr).value_counts() / len(df)

In [ ]:
from sample_features import stacked
# fill daily gaps
stacked_df = stacked.fill_gaps_dates(bar_dates, fill_col='price_vwap')

# stacked_df[['price_high','price_low','price_close', 'price_vwap']].plot_bokeh(sizing_mode="scale_height")

# stacked stats
stats_df = stacked.stacked_df_stats(stacked_df)


In [ ]:
stats_df.describe()